In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

## 人才价值特征还未做

In [2]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_50594/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [3]:
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [4]:
# min_annual_salary和max_annual_salary的缺失值太严重了
jd_salary_columns = ['min_annual_salary', 'max_annual_salary', 'min_month_salary', 'max_month_salary', 'pay_months']

#jd_salary_columns = ['min_month_salary', 'max_month_salary', 'pay_months']
cv_salary_columns = ['currentSalary', 'desiredSalary']

In [5]:
# 是不是因为jd的数据比较规范，所以这里的的salary特征覆盖率很高
all_data[jd_salary_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   min_annual_salary  2758 non-null   float64
 1   max_annual_salary  2758 non-null   float64
 2   min_month_salary   78538 non-null  float64
 3   max_month_salary   78538 non-null  float64
 4   pay_months         78538 non-null  float64
dtypes: float64(5)
memory usage: 3.1 MB


In [8]:
78538/81296

0.9660745916158237

In [6]:
# 从数字上看，月薪的单位是k
all_data[jd_salary_columns]

,min_annual_salary,max_annual_salary,min_month_salary,max_month_salary,pay_months
0,80.0,150.0,NaN,NaN,NaN
1,28.0,35.0,NaN,NaN,NaN
2,25.0,35.0,NaN,NaN,NaN
3,60.0,500.0,NaN,NaN,NaN
4,NaN,NaN,25.0,45.0,12.0
...,...,...,...,...,...
81291,NaN,NaN,10.0,15.0,12.0
81292,NaN,NaN,100.0,120.0,12.0
81293,NaN,NaN,50.0,60.0,13.0
81294,NaN,NaN,15.0,30.0,14.0


In [16]:
# 这个例子的月薪范围[100, 120]，符合预期
all_data.iloc[81292]

cv_id                             8a69d7c282199453018247bac4a36859
jd_id                             8a69c468826ee9b40182831bedfc5e79
jd_code                                     JD20220809225701866333
title                                                       中央市场总监
deadline                                                       NaN
                                       ...                        
avatar                                                         NaN
eduTracks        ['{"schoolTag": "211", "endDate": "1999-07", "...
jobTracks        ['{"endDate": "2022-07", "companyName": "上海微创医...
projectTracks                                   ['{"name": null}']
ds.1                                                      20220821
Name: 81292, Length: 86, dtype: object

In [9]:
# cv的salary覆盖率不到一半啊
all_data[cv_salary_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   currentSalary  35324 non-null  object
 1   desiredSalary  43456 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [11]:
43456/81296

0.5345404447943318

In [13]:
cv_salary_data = all_data[cv_salary_columns]
cv_salary_data

,currentSalary,desiredSalary
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,保密,25 - 30k · 15薪
...,...,...
81291,6000,6千-8千/月
81292,60+期权,100
81293,60+期权,100
81294,25K,28K


### 先解析desiredSalary

In [14]:
# step1 desiredSalary列空值填充
cv_salary_data['desiredSalary'].fillna('', inplace=True)
cv_salary_data['currentSalary'].fillna('', inplace=True)

cv_salary_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81296 entries, 0 to 81295
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   currentSalary  81296 non-null  object
 1   desiredSalary  81296 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_50594/2302065646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_salary_data['desiredSalary'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_50594/2302065646.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_salary_data['currentSalary'].fillna('', inplace=True)


In [15]:
# 其实是有值的，''，看不见而已
cv_salary_data

,currentSalary,desiredSalary
0,,
1,,
2,,
3,,
4,保密,25 - 30k · 15薪
...,...,...
81291,6000,6千-8千/月
81292,60+期权,100
81293,60+期权,100
81294,25K,28K


In [18]:
def get_salary_year_cv(series, salary_col):
    """
    获取年薪的数字,以k为单位
    """
    salary = series[salary_col]

    # 转中文简体
    salary = convert(salary, "zh-hans")
    # 转英文小写
    salary = salary.lower()

    # case 10.5k*15.5
    res = re.search(r'(\d+\.*\d*)([k,w])[\*,x,·, ]+(\d+\.*\d*)', salary)
    if res:
        num1 = float(res.group(1))
        unit = res.group(2)
        num3 = float(res.group(3))
        #print(num1, unit, num3)
        salary_year = int(num1 * num3) 
        return salary_year if unit == "k" else 10 * salary_year
    
    # case 10.5*15.5k
    res = re.search(r'(\d+\.*\d*)[\*,x,·, ]+(\d+\.*\d*)([k,w])', salary)
    if res:
        num1 = float(res.group(1))
        num2 = float(res.group(2))
        unit = res.group(3)
        #print(num1, num2, unit)
        salary_year = int(num1 * num2) 
        return salary_year if unit == "k" else 10 * salary_year

    # case 10万/年
    res = re.search(r'(\d+)([万,w,元])/年', salary)
    if res:
        num = float(res.group(1))
        unit = res.group(2)
        salary_year = int(num * 10) if unit in ['万', 'w'] else int(num / 1000)
        return salary_year
    
    # case 10w/月
    res = re.search(r'(\d+)([万,w,元])/月', salary)
    if res:
        num = float(res.group(1))
        unit = res.group(2)
        # 默认12个月
        salary_year = int(num * 10) * 12 if unit in ['万', 'w'] else int(num / 1000) * 12
        return salary_year
 
    # 30k
    res = re.search(r'(\d+)k', salary)
    if res:
        num = float(res.group(1))
        # 默认12个月
        salary_year = int(num * 12)
        return salary_year
    
    # 42万
    res = re.search(r'(\d+)[万,w]', salary)
    if res:
        num = float(res.group(1))
        salary_year = int(num * 10)
        return salary_year

    # 9000元/月*12月
    res = re.search(r'(\d+)元/月[\*,x,·, ]+(\d+)', salary)
    if res:
        num1 = float(res.group(1))
        num2 = float(res.group(2))
        salary_year = int(num1 * num3 / 1000) 
        return salary_year
    
    # 都没匹配上
    return -1

cv_salary_data['parsed_desiredSalary'] = cv_salary_data.apply(get_salary_year_cv, axis=1, args=('desiredSalary', ))
cv_salary_data['parsed_currentSalary'] = cv_salary_data.apply(get_salary_year_cv, axis=1, args=('currentSalary', ))

cv_salary_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_50594/4131299421.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_salary_data['parsed_desiredSalary'] = cv_salary_data.apply(get_salary_year_cv, axis=1, args=('desiredSalary', ))
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_50594/4131299421.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_salary_data['parsed_currentSalary'] = cv_salary_data.apply(get_salary_year_cv, axis=1, args=('currentSalary', ))


,currentSalary,desiredSalary,parsed_desiredSalary,parsed_currentSalary
0,,,-1,-1
1,,,-1,-1
2,,,-1,-1
3,,,-1,-1
4,保密,25 - 30k · 15薪,450,-1
...,...,...,...,...
81291,6000,6千-8千/月,-1,-1
81292,60+期权,100,-1,-1
81293,60+期权,100,-1,-1
81294,25K,28K,336,300


In [17]:
# parsed_desiredSalary的覆盖率
parsed_num = 0
for index, salary in enumerate(cv_salary_data['parsed_desiredSalary']):
    if salary == -1:
        continue
    parsed_num += 1
    # 看看是哪些行的salary被解析出来了
    # print('{}: {}'.format(index, salary))

all_num = 81296
print('parsed_desiredSalary的覆盖率: ', parsed_num / all_num)

# parsed_currentSalary的覆盖率
parsed_num = 0
for index, salary in enumerate(cv_salary_data['parsed_currentSalary']):
    if salary == -1:
        continue
    parsed_num += 1
    # 看看是哪些行的salary被解析出来了
    # print('{}: {}'.format(index, salary))

# parsed_desiredSalary的覆盖率
all_num = 81296
print('parsed_currentSalary的覆盖率: ', parsed_num / all_num)

parsed_desiredSalary的覆盖率:  0.3526618775831529
parsed_currentSalary的覆盖率:  0.33888506199567014


0.3526618775831529

In [58]:
cv_salary_data.iloc[129]

currentSalary           25000~35000元/月
desiredSalary           35000-50000元/月
parsed_desiredSalary               600
parsed_currentSalary               420
Name: 129, dtype: object

In [59]:
35*12

420